In [36]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [37]:
rent_obj = []
all_property = []

for page in range(21,29):
    link = 'https://www.apartments.com/Townhomes/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='prosumer placard')

    # Loop through returned results and get the hyperlink of the properties of the page
    for result in results:
        find_tag = result.find('a')['href']
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
        
        full_address = result.find('div', class_ = "location").text
        try:
            city = new_results.find('div', class_ = "propertyAddress").select('span')[0].text
        except:
            city = 'null'
        try:
            state = new_results.find('div', class_ = "propertyAddress").select('span')[1].text
        except:
            state = 'null'
        zip_code = new_results.find('div', class_ = "propertyAddress").select('span')[2].text
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('table', class_="availabilityTable")
        try:
            bath_number = house_details.find('td', class_="baths").find('span', class_="shortText").text.replace('\r\n                    ', "").strip()
        except:
            bath_number = 'null'
        house_square_feet = house_details.find('td', class_="sqft").text
        square_feet = house_square_feet.split(" ")[0];
        lease_length = house_details.find('td', class_="leaseLength").text
        availability_check = house_details.find('td', class_="available").text.replace('\r\n                    ', "").strip()
        all_the_rents = bedroom_container.findAll('span', class_="rentRollup")

        bedroom_info = []
        for rent_block in all_the_rents:
            bedroom_type = rent_block.find('span', class_="shortText").text
            try:
                price = re.search('\$.* ', rent_block.text)
                price = price.group(0)[:-3]

            except:
                if "Call for Rent" in rent_block.text:
                    price = ""
                else:
                    price = re.search('\$.*', rent_block.text).group()[:-1]
            bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
        rent_obj = {
                "City_Rent": city,
                "Type": "Townhomes",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                "Full_Address": full_address,
                "Rent_link": find_tag,
                "Bath_Info": bath_number,
                "Square_Feet_Data": house_square_feet,
                "Square_Feet": square_feet,
                "Lease_length": lease_length,
                "Availability_check": availability_check
            }
        for room_type in bedroom_info:
            rent_obj[room_type["bedroom_type"]] = room_type["price"];
        
        all_property.append(rent_obj)
        time.sleep(1)
print(all_property)

[{'City_Rent': '3258 Hilltop Dr', 'Type': 'Townhomes', 'State_Rent': 'Chino Hills', 'Zip_code': 'CA', 'Bedroom_Info': [{'bedroom_type': '3 Beds', 'price': '$2,300'}], 'Full_Address': 'Chino Hills, CA 91709', 'Rent_link': 'https://www.apartments.com/3258-hilltop-dr-chino-hills-ca/85t2dm2/', 'Bath_Info': '3 BAs', 'Square_Feet_Data': '1,411 Sq Ft', 'Square_Feet': '1,411', 'Lease_length': '', 'Availability_check': 'Available Now', '3 Beds': '$2,300'}, {'City_Rent': '33963 Cape Cove', 'Type': 'Townhomes', 'State_Rent': 'Dana Point', 'Zip_code': 'CA', 'Bedroom_Info': [{'bedroom_type': '2 Beds', 'price': '$6,000'}], 'Full_Address': 'Dana Point, CA 92629', 'Rent_link': 'https://www.apartments.com/33963-cape-cove-dana-point-ca/tp8088b/', 'Bath_Info': '2 BAs', 'Square_Feet_Data': '1,350 Sq Ft', 'Square_Feet': '1,350', 'Lease_length': '', 'Availability_check': 'Available Now', '2 Beds': '$6,000'}, {'City_Rent': '1762 Coolidge Ln', 'Type': 'Townhomes', 'State_Rent': 'Placentia', 'Zip_code': 'CA', 

In [38]:
rent_data = pd.DataFrame(all_property)
rent_data

,1 Bed,2 Beds,3 Beds,4 Beds,4+ Beds,Availability_check,Bath_Info,Bedroom_Info,City_Rent,Full_Address,Lease_length,Rent_link,Square_Feet,Square_Feet_Data,State_Rent,Type,Zip_code
0,NaN,NaN,"$2,300",NaN,NaN,Available Now,3 BAs,"[{'bedroom_type': '3 Beds', 'price': '$2,300'}]",3258 Hilltop Dr,"Chino Hills, CA 91709",,https://www.apartments.com/3258-hilltop-dr-chi...,"1,411","1,411 Sq Ft",Chino Hills,Townhomes,CA
1,NaN,"$6,000",NaN,NaN,NaN,Available Now,2 BAs,"[{'bedroom_type': '2 Beds', 'price': '$6,000'}]",33963 Cape Cove,"Dana Point, CA 92629",,https://www.apartments.com/33963-cape-cove-dan...,"1,350","1,350 Sq Ft",Dana Point,Townhomes,CA
2,NaN,NaN,"$3,000",NaN,NaN,Available Now,3 BAs,"[{'bedroom_type': '3 Beds', 'price': '$3,000'}]",1762 Coolidge Ln,"Placentia, CA 92870",,https://www.apartments.com/1762-coolidge-ln-pl...,"1,726","1,726 Sq Ft",Placentia,Townhomes,CA
3,NaN,"$4,695",NaN,NaN,NaN,Available Now,2 BAs,"[{'bedroom_type': '2 Beds', 'price': '$4,695'}]",26 Vienna,"Newport Beach, CA 92660",,https://www.apartments.com/26-vienna-newport-b...,"2,002","2,002 Sq Ft",Newport Beach,Townhomes,CA
4,"$1,395",NaN,NaN,NaN,NaN,Available Now,1 BA,"[{'bedroom_type': '1 Bed', 'price': '$1,395'}]",11814 Newgate Ave,"Whittier, CA 90605",,https://www.apartments.com/11814-newgate-ave-w...,335,335 Sq Ft,Whittier,Townhomes,CA
5,NaN,"$1,700",NaN,NaN,NaN,Available Now,2 BAs,"[{'bedroom_type': '2 Beds', 'price': '$1,700'}]",7419 Saddle Rd,"Rancho Cucamonga, CA 91730",,https://www.apartments.com/7419-saddle-rd-ranc...,"1,008","1,008 Sq Ft",Rancho Cucamonga,Townhomes,CA
6,NaN,"$1,450",NaN,NaN,NaN,Available Now,1 BA,"[{'bedroom_type': '2 Beds', 'price': '$1,450'}]",180 E 7th St,"Perris, CA 92570",,https://www.apartments.com/180-e-7th-st-perris...,"1,364","1,364 Sq Ft",Perris,Townhomes,CA
7,NaN,"$8,000",NaN,NaN,NaN,Available Now,2 BAs,"[{'bedroom_type': '2 Beds', 'price': '$8,000'}]",35067 Beach Rd,"Dana Point, CA 92624",,https://www.apartments.com/35067-beach-rd-dana...,"1,500","1,500 Sq Ft",Dana Point,Townhomes,CA
8,"$9,875",NaN,NaN,NaN,NaN,Available Now,1 BA,"[{'bedroom_type': '1 Bed', 'price': '$9,875'}]",239 Fairview St,"Laguna Beach, CA 92651",,https://www.apartments.com/239-fairview-st-lag...,"1,040","1,040 Sq Ft",Laguna Beach,Townhomes,CA
9,NaN,"$3,200",NaN,NaN,NaN,Available Now,3 BAs,"[{'bedroom_type': '2 Beds', 'price': '$3,200'}]",167 Neptune,"Irvine, CA 92618",,https://www.apartments.com/167-neptune-irvine-...,"1,798","1,798 Sq Ft",Irvine,Townhomes,CA


In [39]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q1-2020_Apartment.com_Townhomes_Data_page21-28.csv", index=False, header=True)